In [60]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [62]:
!pip install surprise

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from surprise import Reader, Dataset, SVD
import surprise
import warnings; warnings.simplefilter('ignore')


In [64]:
movie_data = pd. read_csv('/content/drive/My Drive/Colab Notebooks_ML_Dataset/movies_metadata.csv')
movie_data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [0]:
movie_data['genres'] = movie_data['genres'].fillna('[]').apply(literal_eval).apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [0]:
movie_data['year'] = pd.to_datetime(movie_data['release_date'], errors='coerce').apply(lambda x: str(x).split('-')[0] if x != np.nan else np.nan)

In [0]:
movie_credits = pd.read_csv('/content/drive/My Drive/Colab Notebooks_ML_Dataset/credits.csv')
movie_keywords = pd.read_csv('/content/drive/My Drive/Colab Notebooks_ML_Dataset/keywords.csv')

In [0]:
movie_data = movie_data.drop([19730, 29503, 35587])



In [0]:
movie_keywords['id'] = movie_keywords['id'].astype('int')
movie_credits['id'] = movie_credits['id'].astype('int')
movie_data['id'] = movie_data['id'].astype('int')

In [70]:
movie_data.shape

(45463, 25)

In [0]:
movie_data = movie_data.merge(movie_credits, on='id')
movie_data = movie_data.merge(movie_keywords, on='id')

In [0]:
movie_links_small = pd.read_csv('/content/drive/My Drive/Colab Notebooks_ML_Dataset/links_small.csv')
movie_links_small = movie_links_small[movie_links_small['tmdbId'].notnull()]['tmdbId'].astype('int')

In [73]:
small_md = movie_data[movie_data['id'].isin(movie_links_small)]
small_md.shape

(9219, 28)

In [0]:
small_md['cast'] = small_md['cast'].apply(literal_eval)
small_md['crew'] = small_md['crew'].apply(literal_eval)
small_md['keywords'] = small_md['keywords'].apply(literal_eval)
small_md['cast_size'] = small_md['cast'].apply(lambda x: len(x))
small_md['crew_size'] = small_md['crew'].apply(lambda x: len(x))

In [0]:
def give_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [0]:
small_md['director'] = small_md['crew'].apply(give_director)

In [0]:
small_md['cast'] = small_md['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
small_md['cast'] = small_md['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

In [0]:
small_md['keywords'] = small_md['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

In [0]:
small_md['cast'] = small_md['cast'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [0]:
small_md['director'] = small_md['director'].astype('str').apply(lambda x: str.lower(x.replace(" ", "")))
small_md['director'] = small_md['director'].apply(lambda x: [x,x, x])

In [0]:
series = small_md.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
series.name = 'keyword'

In [82]:
series = series.value_counts()
series[:5]

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
Name: keyword, dtype: int64

In [83]:
series

independent film        610
woman director          550
murder                  399
duringcreditsstinger    327
based on novel          318
                       ... 
wings                     1
heart trouble             1
socially awkward          1
stolen jewellery          1
snoopy                    1
Name: keyword, Length: 12940, dtype: int64

In [0]:
series = series[series > 1]

In [85]:
stemmer = SnowballStemmer('english')
stemmer.stem('dogs')

'dog'

In [0]:
def find_keywords(x):
    words = []
    for i in x:
        if i in series:
            words.append(i)
    return words

In [0]:
movie_data['keywords'] = small_md['keywords'].apply(find_keywords)
small_md['keywords'] = small_md['keywords'].apply(lambda x: [stemmer.stem(i) for i in x])
small_md['keywords'] = small_md['keywords'].apply(lambda x: [str.lower(i.replace(" ", "")) for i in x])

In [0]:
small_md['soup'] = small_md['keywords'] + small_md['cast'] + small_md['director'] +small_md['genres']
small_md['soup'] = small_md['soup'].apply(lambda x: ' '.join(x))

In [0]:
counter = CountVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
counter_matrix = counter.fit_transform(small_md['soup'])

In [0]:
cosine_similar = cosine_similarity(counter_matrix, counter_matrix)

In [0]:
small_md = small_md.reset_index()
titles = small_md['title']
indices = pd.Series(small_md.index, index=small_md['title'])

In [0]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan

In [0]:
id_map = pd.read_csv('/content/drive/My Drive/Colab Notebooks_ML_Dataset/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(small_md[['title', 'id']], on='id').set_index('title')
#id_map = id_map.set_index('tmdbId')

In [0]:
indices_map = id_map.set_index('id')

In [95]:
ratings = pd.read_csv('/content/drive/My Drive/Colab Notebooks_ML_Dataset/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,110,1.0,1425941529
1,1,147,4.5,1425942435
2,1,858,5.0,1425941523
3,1,1221,5.0,1425941546
4,1,1246,5.0,1425941556


In [0]:
reader=Reader()

In [0]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)




In [0]:
svd=SVD()

In [102]:
trainset = data.build_full_trainset()
print(trainset)
svd.fit(trainset)

In [0]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = id_map.loc[title]['id']
    #print(idx)
    movie_id = id_map.loc[title]['movieId']
    
    sim_scores = list(enumerate(cosine_similar[int(idx)]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:26]
    movie_indices = [i[0] for i in sim_scores]
    
    movies = small_md.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'year', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(20)


In [114]:
hybrid(2, 'Avatar')

,title,vote_count,vote_average,year,id,est
8865,Star Wars: The Force Awakens,7993.0,7.5,2015,140607,4.099147
8401,Star Trek Into Darkness,4479.0,7.4,2013,54138,3.778777
2014,Fantastic Planet,140.0,7.6,1973,16306,3.695507
922,The Abyss,822.0,7.1,1989,2756,3.557302
8658,X-Men: Days of Future Past,6155.0,7.5,2014,127585,3.484056
974,Aliens,3282.0,7.7,1986,679,3.358708
522,Terminator 2: Judgment Day,4274.0,7.7,1991,280,3.331779
1011,The Terminator,4208.0,7.4,1984,218,3.301727
1376,Titanic,7770.0,7.5,1997,597,3.209748
1621,Darby O'Gill and the Little People,35.0,6.7,1959,18887,3.200993
